# Introduction

- Following code is for retrieving data of iitk students from the iitk server using their roll number. <br>
- In roll_no_list, append all the roll numbers whose name and email you want to retrieve from the server. <br>
- If you are outside from IIT Kanpur, then don't forget to use ssl-vpn proxy.
<br><br>
Change Gmail settings - https://myaccount.google.com/lesssecureapps

## Import Libraries

In [1]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
import requests
import re
import pandas as pd

import email
import smtplib
import ssl

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [16]:
# Save roll numbers in roll_no_list

roll_no_list = []

# for y19
for i in range(1000):
    roll = 190001 + i
    roll_no_list.append(roll)

roll_no_list.append(15807678)

## Extracting Data from IITK Server

In [17]:
s = requests.Session()
s.get("https://oa.cc.iitk.ac.in/Oa/Jsp/Main_Frameset.jsp")
s.get("https://oa.cc.iitk.ac.in/Oa/Jsp/Main_Intro.jsp?frm='SRCH'")
s.get("https://oa.cc.iitk.ac.in/Oa/Jsp/OAServices/IITK_Srch.jsp?typ=stud")


headers1 = {
    "Referer": "https://oa.cc.iitk.ac.in/Oa/Jsp/OAServices/IITk_SrchStudRoll_new.jsp"
}

payload1 = {
    'typ': ['stud'] * 12,
    'numtxt': '',
    'sbm': ['Y'] * 12
}

In [1]:
record = []
# roll_no_list = [15807678]
for roll in roll_no_list:
    payload1['numtxt'] = roll
    r1 = s.post("https://oa.cc.iitk.ac.in/Oa/Jsp/OAServices/IITk_SrchRes_new.jsp", headers=headers1, data=payload1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')

    name = ''
    email = ''
    department = ''
    gender = ''

    for para in soup1.select('.TableContent p'):
        body = para.get_text().strip()
        field = body.split(':')
        key = field[0].strip()
        value = field[1].strip()
        if key == 'Name':
            name = value.lower().title()
        elif key == 'E-Mail':
            email = value
        elif key == 'Department':
            department = value
        elif key == 'Gender':
            gender = value[0]

    if name != "" and email != "" and email != "Not Available":
        data = [name, email, department, gender, roll]
        record.append(data)
        user_name = name.split(" ")[0]
#         print (name, email, department, gender, roll)

df = pd.DataFrame(record, columns = ['Name', 'Email', 'Department', 'Gender', 'Roll No']) 
df.to_csv('y19_students.csv', encoding='utf-8', index=False)


# Sending mail to recipients

In [9]:
def message_template(user_name):
    message = """\
    
    
    <html>
    <body link="#00a5b5" vlink="#00a5b5" alink="#00a5b5" style="margin-top: 20px; margin-bottom: 20px;">
    <div class="s12">
        <table class=" main contenttable" align="center" style="font-weight: normal;border-collapse: collapse;border: 0;margin-left: auto;margin-right: auto;padding: 0;font-family: Arial, sans-serif;color: #555559;background-color: white;font-size: 16px;line-height: 26px;width: 600px;">
            <tr>
                <td class="border" style="border-collapse: collapse;border: 1px solid #eeeff0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 26px;">
                    <table style="font-weight: normal;border-collapse: collapse;border: 0;margin: 0;padding: 0;font-family: Arial, sans-serif;">
                        <tr>
                            <td colspan="4" valign="top" class="image-section" style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #fff;font-family: Arial, sans-serif;font-size: 20px;line-height: 26px;background: linear-gradient(45deg, #1de099, #1dc8cd);border-bottom: 0px solid #00a5b5">
                                <h3 style="margin: 20px; text-align: center;">UnchaAI</h3>
                            </td>
                        </tr>
                        <tr>
                            <td valign="top" class="side title" style="border-collapse: collapse;border: 0;margin: 0;padding: 20px;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 26px;vertical-align: top;background-color: white;border-top: none;">
                                <table style="font-weight: normal;border-collapse: collapse;border: 0;margin: 0;padding: 0;font-family: Arial, sans-serif;">
                                    <tr>
                                        <td class="head-title" style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 28px;line-height: 34px;font-weight: bold; text-align: center;">
                                            <div class="mktEditable" id="main_title">
                                                Paid Internship at UnchaAI
                                            </div>
                                        </td>
                                    </tr>
                                    <tr>
                                        <td class="top-padding" style="border-collapse: collapse;border: 0;margin: 0;padding: 15px 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 21px;">
                                            <hr size="1" color="#eeeff0">
                                        </td>
                                    </tr>
                                    <tr>
                                        <td class="text" style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 26px;">
                                        <div class="mktEditable" id="main_text">
                                            Hi {name},<br><br>
                                            We are an early-stage EdTech Startup founded by IITians and NITians. We at UnchaAI (https://reynsiatech.github.io) are trying to boost the performance of JEE-aspirants by providing them with personal mentorship support. Currently, we are looking for interns who have the passion and zeal to guide JEE-aspirants. The main funda is, you made many mistakes while preparing for JEE, and currently, many other aspirants will make the same mistakes and decrease their chances of getting selected into IIT-JEE. Your experience and knowledge can help many others and without putting in much effort.<br><br>
                                            If interested, please fill the form - <a href="https://forms.gle/emrcUy64NqL6Eukc7">https://forms.gle/emrcUy64NqL6Eukc7</a><br><br>
                                            <strong>About Internship</strong><br>The internship will be paid and work from home.<br><strong>Duration:</strong> Available at least for two months (If your semester starts in between, then we’ll adjust term accordingly)<br><strong>Stipend:</strong> Stipend will be varied and mainly depend on your time commitment and performance. Net Stipend will be (2500 + 500 (bonus)) Rs/month minimum 1 hr commitment everyday. If you can give more time, then your stipend will increase accordingly.<br><br>
                                            <br><br><strong></strong><strong></strong>
                                            <strong>Responsibilities</strong><br>Guide and mentor JEE aspirants, by solving their doubts and interacting with them<br><br>
                                            <strong>Prerequisites</strong><br>Must have good knowledge of Physics, Chemistry and Mathematics up to JEE-Advanced level and passion for helping someone<br><br>
                                            If interested, please fill the form - <a href="https://forms.gle/emrcUy64NqL6Eukc7">https://forms.gle/emrcUy64NqL6Eukc7</a><br><br>
                                            <strong>Benefits</strong><br>                                            
                                            1. Stipend + Bonus<br>
                                            2. Internship Certificate<br>
                                            3. Get the opportunity to work and learn with a growing startup<br>
                                            4. Increase your network<br>
                                            5. A long term association with Entrepreneurs<br><br>
                                            We have an immediate requirement and limited openings so the interview will be on a first-come, first-serve basis. So please don’t wait and fill the form ASAP. We will take submission until 18th July.<br><br>
                                            For any queries, feel free to contact:<br>Piyush Patel - +91 70071 88949, dapiyushpatel@gmail.com<br><br>
                                            Regards,<br>
                                            <a href="https://reynsiatech.github.io">Team UnchaAI</a><br>
                                        </div>
                                        </td>
                                    </tr>
                                    <tr>
                                        <td style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 24px;">
                                         &nbsp;<br>
                                        </td>
                                    </tr>
                                </table>
                            </td>
                        </tr>
                        <tr bgcolor="#fff" style="border-top: 4px solid #00a5b5;">
                            <td valign="top" class="footer" style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 26px;background: #fff;text-align: center;">
                                <table style="font-weight: normal;border-collapse: collapse;border: 0;margin: 0;padding: 0;font-family: Arial, sans-serif;">
                                    <tr>
                                        <td class="inside-footer" align="center" valign="middle" style="border-collapse: collapse;border: 0;margin: 0;padding: 20px;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 12px;line-height: 16px;vertical-align: middle;text-align: center;width: 580px;">
                                        <div id="address" class="mktEditable">
                                            <b>UnchaAI</b><br>
                                            Reynsia Tech Pvt Ltd.<br>
                                            <a style="color: #00a5b5;" href="https://reynsiatech.github.io/">Contact Us</a>
                                        </div>
                                        </td>
                                    </tr>
                                </table>
                            </td>
                        </tr>
                    </table>
                </td>
            </tr>
        </table>
    </div>
    </body>
    </html>
    
    
    """.format(name = user_name)
    
    return message

In [8]:
message = message_template("Shivam Pal")

In [5]:
def send_mail(user_name, user_mail_id):

    email_admin = 'reynsiatech@gmail.com'
    admin_password = 'Unchaaikijeetho'

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_admin, admin_password)

    msg = MIMEMultipart()
    msg['From'] = "Reynsia Tech"
    msg['To'] = user_mail_id
    msg['Subject'] = "Work from home Internship Opportunity"

    message = message_template(user_name)

    msg.attach(MIMEText(message, 'html'))
    text = msg.as_string()
    server.sendmail(email_admin, user_mail_id, text)

    server.quit()

In [7]:
user_name = "Shivam Pal"
email = "shivam.pal128@gmail.com"
send_mail(user_name, email)

user_name = "Shivam Pal"
email = "shivampa@iitk.ac.in"
send_mail(user_name, email)

user_name = "Deepak Kumar"
email = "dk24pk@gmail.com"
send_mail(user_name, email)

user_name = "Piyush Patel"
email = "dapiyushpatel@gmail.com"
send_mail(user_name, email)

## Sending mails to Students

In [34]:
df = pd.read_csv("y19_students.csv")
df_civil = df[df["Department"] == "CIVIL ENGG."].reset_index()[:50]              # No = 122
df_chem = df[df["Department"] == "CHEMICAL ENGG."].reset_index()[:50]            # No = 87

In [35]:
df_civil

,index,Name,Email,Department,Gender,Roll No
0,2,Aadeesh Jain,aadeej@iitk.ac.in,CIVIL ENGG.,M,190003
1,8,Aanchal Jain,aanchal@iitk.ac.in,CIVIL ENGG.,F,190009
2,18,Abhaya Pratap Singh,asinghp@iitk.ac.in,CIVIL ENGG.,M,190019
3,26,Abhinav Kumar,abhiku@iitk.ac.in,CIVIL ENGG.,M,190027
4,29,Abhinav Raj Singh,abhinavr@iitk.ac.in,CIVIL ENGG.,M,190030
5,33,Abhishek,abhitam@iitk.ac.in,CIVIL ENGG.,M,190034
6,42,Abhishek Vijay Yadav,abhivy@iitk.ac.in,CIVIL ENGG.,M,190043
7,70,Agarwal Aryan Rajesh,aryanag@iitk.ac.in,CIVIL ENGG.,M,190071
8,99,Alok Kumar Thakur,alokthak@iitk.ac.in,CIVIL ENGG.,M,190100
9,104,Aman Jain,amanjain@iitk.ac.in,CIVIL ENGG.,M,190105


In [36]:
df_chem

,index,Name,Email,Department,Gender,Roll No
0,10,Aarushi Singh,aarushis@iitk.ac.in,CHEMICAL ENGG.,F,190011
1,15,Abhay Dayal Mathur,admathur@iitk.ac.in,CHEMICAL ENGG.,M,190016
2,20,Abhijeet Sharma,abhijdev@iitk.ac.in,CHEMICAL ENGG.,M,190021
3,61,Aditya Jindal,jaditya@iitk.ac.in,CHEMICAL ENGG.,M,190062
4,90,Akshat Srivastava,akshtsri@iitk.ac.in,CHEMICAL ENGG.,M,190091
5,97,Alok Kumar (B Prasad),kralok@iitk.ac.in,CHEMICAL ENGG.,M,190098
6,114,Amit Kumar,amkumar@iitk.ac.in,CHEMICAL ENGG.,M,190115
7,120,Amrapali Santosh Pawar,amrapali@iitk.ac.in,CHEMICAL ENGG.,F,190121
8,123,Anant Dev,anantdev@iitk.ac.in,CHEMICAL ENGG.,M,190124
9,128,Ananyae Kumar Bhartari,ananyae@iitk.ac.in,CHEMICAL ENGG.,M,190129


In [41]:
# Sending mails to civil eng. students
# for i in range(50):
#     name = df_civil[["Name"]].iloc[i].values[0]
#     email = df_civil[["Email"]].iloc[i].values[0]
#     print(name, email)
#     send_mail(name, email)

Aadeesh  Jain aadeej@iitk.ac.in
Aanchal  Jain aanchal@iitk.ac.in
Abhaya Pratap Singh asinghp@iitk.ac.in
Abhinav  Kumar abhiku@iitk.ac.in
Abhinav Raj Singh abhinavr@iitk.ac.in
Abhishek abhitam@iitk.ac.in
Abhishek Vijay Yadav abhivy@iitk.ac.in
Agarwal Aryan Rajesh aryanag@iitk.ac.in
Alok Kumar Thakur alokthak@iitk.ac.in
Aman  Jain amanjain@iitk.ac.in
Amit Kumar Yadav amitkum@iitk.ac.in
Anay  Nagar anayn@iitk.ac.in
Aniket  Sankhi sankhi@iitk.ac.in
Animesh  Singh animesh@iitk.ac.in
Ansh  Saxena anshsx@iitk.ac.in
Anshul  Agarwal anshulag@iitk.ac.in
Archana archanay@iitk.ac.in
Arush aarush@iitk.ac.in
Aryash  Pateriya aryashp@iitk.ac.in
Ashish ashishce@iitk.ac.in
Ashok Kumar Jyani ashokjy@iitk.ac.in
Ashwani Kumar Srivastava ashwanis@iitk.ac.in
Atul  Umak atulumak@iitk.ac.in
Avadhi  Jindal avadhi@iitk.ac.in
Ayush  Dwivedi ayushvns@iitk.ac.in
Azad  Prajapat azadp@iitk.ac.in
Chandra Shekhar cshekhar@iitk.ac.in
Chirag  Sharma chiragsh@iitk.ac.in
Darshana Singh Baghel darshana@iitk.ac.in
Deepalok 

In [42]:
# Sending mails to chemical eng. students
# for i in range(50):
#     name = df_chem[["Name"]].iloc[i].values[0]
#     email = df_chem[["Email"]].iloc[i].values[0]
#     print(name, email)
#     send_mail(name, email)

Aarushi  Singh aarushis@iitk.ac.in
Abhay Dayal Mathur admathur@iitk.ac.in
Abhijeet  Sharma abhijdev@iitk.ac.in
Aditya  Jindal jaditya@iitk.ac.in
Akshat  Srivastava akshtsri@iitk.ac.in
Alok Kumar (B Prasad) kralok@iitk.ac.in
Amit  Kumar amkumar@iitk.ac.in
Amrapali Santosh Pawar amrapali@iitk.ac.in
Anant  Dev anantdev@iitk.ac.in
Ananyae Kumar Bhartari ananyae@iitk.ac.in
Aniket  Saini saniket@iitk.ac.in
Anmol  Agarwal anmolag@iitk.ac.in
Avinash  Bansal avinashb@iitk.ac.in
Ayush  Baranwal ayubaran@iitk.ac.in
Bhaskar  Shukla bshukla@iitk.ac.in
Bhatt  Harshal harshal@iitk.ac.in
Darpan darpanar@iitk.ac.in
Deborathi  Sen debosen@iitk.ac.in
Dhananjay  Gupta gdhanjay@iitk.ac.in
Dheeraj  Agarwal adheeraj@iitk.ac.in
Diksha  Banka diksha@iitk.ac.in
Dinesh dineshkm@iitk.ac.in
Divyansh  Tripathi divtri@iitk.ac.in
Divyanshi  Agarwal divi@iitk.ac.in
Gaurav Kumar Sharma gauravs@iitk.ac.in
Harshit  Balwada hbalwada@iitk.ac.in
Ishaan  Sodhani ishaans@iitk.ac.in
Jitin jitin@iitk.ac.in
Kanika  Kukreja kanik